## testing sft formatting

In [1]:
from datasets import load_from_disk

/Users/matthewho/miniconda3/envs/gfn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_from_disk("../data/test_nns")

In [3]:
train = dataset["train"]
test = dataset["test"]

In [5]:
type(train)

datasets.arrow_dataset.Dataset

In [6]:
train[0]

{'prompt': 'Given the Lean 4 tactic state, suggest a next tactic.\n\nTactic state:\n---\nα : Type u_1\nV : Type u_2\nP : Type u_3\nW : Type u_4\nQ : Type u_5\ninst✝⁵ : SeminormedAddCommGroup V\ninst✝⁴ : PseudoMetricSpace P\ninst✝³ : NormedAddTorsor V P\ninst✝² : NormedAddCommGroup W\ninst✝¹ : MetricSpace Q\ninst✝ : NormedAddTorsor W Q\np₁ p₂ p₃ p₄ : P\n⊢ ↑(nndist (p₁ -ᵥ p₂) (p₃ -ᵥ p₄)) ≤ ↑(nndist p₁ p₃) + ↑(nndist p₂ p₄)\n---\nNext tactic:\n---\n',
 'completion': 'norm_cast'}

In [7]:
train.features

{'prompt': Value(dtype='string', id=None),
 'completion': Value(dtype='string', id=None)}

In [8]:
from datasets import Dataset

In [9]:
dummy = Dataset.from_dict({"text": ["hello", "world"]})

In [10]:
type(dummy)

datasets.arrow_dataset.Dataset

In [11]:
isinstance(dummy, Dataset)

True

In [12]:
isinstance(train, Dataset)

True

In [4]:
print(train[0]["prompt"])

Given the Lean 4 tactic state, suggest a next tactic.

Tactic state:
---
α : Type u_1
V : Type u_2
P : Type u_3
W : Type u_4
Q : Type u_5
inst✝⁵ : SeminormedAddCommGroup V
inst✝⁴ : PseudoMetricSpace P
inst✝³ : NormedAddTorsor V P
inst✝² : NormedAddCommGroup W
inst✝¹ : MetricSpace Q
inst✝ : NormedAddTorsor W Q
p₁ p₂ p₃ p₄ : P
⊢ ↑(nndist (p₁ -ᵥ p₂) (p₃ -ᵥ p₄)) ≤ ↑(nndist p₁ p₃) + ↑(nndist p₂ p₄)
---
Next tactic:
---



In [11]:
print(train[0]["completion"])

norm_cast


In [15]:
wns = load_from_disk("../data/test_wns")
train_wns = wns["train"]
test_wns = wns["test"]

In [19]:
print(train_wns[0]["prompt"])

Given the Lean 4 tactic state, suggest a next tactic.

Tactic state:
---
l : Type u_1
m : Type u_2
n : Type u_3
o : Type u_4
p : Type u_5
q : Type u_6
m' : o → Type u_7
n' : o → Type u_8
p' : o → Type u_9
R : Type u_10
S : Type u_11
α : Type u_12
β : Type u_13
inst✝² : DecidableEq l
inst✝¹ : DecidableEq m
inst✝ : Zero α
v : l ⊕ m → α
⊢ (of fun i j => diagonal v (Sum.inr i) (Sum.inr j)) = diagonal fun i => v (Sum.inr i)
---
Next tactic:
---



In [20]:
print(train_wns[0]["completion"])

funext i j
---
Resulting state:
---
case h.h
l : Type u_1
m : Type u_2
n : Type u_3
o : Type u_4
p : Type u_5
q : Type u_6
m' : o → Type u_7
n' : o → Type u_8
p' : o → Type u_9
R : Type u_10
S : Type u_11
α : Type u_12
β : Type u_13
inst✝² : DecidableEq l
inst✝¹ : DecidableEq m
inst✝ : Zero α
v : l ⊕ m → α
i j : m
⊢ of (fun i j => diagonal v (Sum.inr i) (Sum.inr j)) i j = diagonal (fun i => v (Sum.inr i)) i j



## fixing the chat template problem
so I get the following error with SFTTrainer


In [14]:
def sft_formatting_func(example):
    return f"{example['prompt']}{example['completion']}"

def sft_formatting_func_batch(examples):
    return [
        f"{example['prompt']}{example['completion']}"
        for example in examples
    ]

In [15]:
test

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 5
})

In [17]:
def tmf1(example):
    concat = example["prompt"] + example["completion"]
    example["text"] = concat
    return example

In [18]:
tm1 = test.map(tmf1)

Map: 100%|██████████| 5/5 [00:00<00:00, 165.79 examples/s]


In [21]:
# tm1[0]
print(tm1[0]["text"])

Given the Lean 4 tactic state, suggest a next tactic.

Tactic state:
---
case h.h
l : Type u_1
m : Type u_2
n : Type u_3
o : Type u_4
p : Type u_5
q : Type u_6
m' : o → Type u_7
n' : o → Type u_8
p' : o → Type u_9
R : Type u_10
S : Type u_11
α : Type u_12
β : Type u_13
inst✝² : DecidableEq l
inst✝¹ : DecidableEq m
inst✝ : Zero α
v : l ⊕ m → α
i j : m
⊢ of (fun i j => diagonal v (Sum.inr i) (Sum.inr j)) i j = diagonal (fun i => v (Sum.inr i)) i j
---
Next tactic:
---
simp only [ne_eq, Sum.inr.injEq, of_apply, diagonal_apply]


In [23]:
tm2 = test.map(tmf1, batched=True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]


ArrowInvalid: Column 2 named text expected length 5 but got length 10

In [25]:
def tmf2(examples):
    print(type(examples))
    return [{} for _ in examples]

In [26]:
tm2 = test.map(tmf2, batched=True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

<class 'datasets.formatting.formatting.LazyBatch'>


TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'list'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.